In [123]:
import numpy as np
import string

In [124]:
np.random.seed(1234)

In [125]:
initial = {}
first_order = {}
second_order = {}

In [126]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [127]:
def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [128]:
for line in open('../documents_for_course/Robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split()

    line_length = len(tokens)
    for count in range(line_length):
        # print(count)
        word = tokens[count]
        # print(word)
        if count==0:
            # If its the first word add a 0 if its not or add one if its there
            initial[word] = initial.get(word,0.) + 1
        else:
            # If it's not the first word take the previous one
            t_1 = tokens[count-1]
            if count == line_length - 1:
                # If its the last word it's added to the dict but it does not have the probability
                add2dict(second_order, (t_1,word),'END')
            if count ==1:
                # If it's the second word we add to the dictionary first_order the previous word as a key and the word as a value
                add2dict(first_order,t_1,word)
            else:
                # If it's not the second word nor the first nor the last we add to the dictionary second_order the previous two words in a tuple format as a key and the word as a value
                t_2 = tokens[count-2]
                add2dict(second_order, (t_2,t_1),word)

In [129]:
#We calculate the probabilities for the initial state vectors
initial_total = sum(initial.values())
for t,c in initial.items():
    initial[t] = c / initial_total

In [130]:
def list2pdict(transition):
    # This function creates a dictionary from a list making the counts for wach word and finally dividing by the total size of the possible words in the tranistion
    #.items() returns the key value pairs as a tuple 
    dictionary = {}
    n_size = len(transition)
    for term in transition:
        dictionary[term] = dictionary.get(term,0.) + 1
    for term,counter in dictionary.items():
        dictionary[term] = counter/ n_size
    return dictionary

In [131]:
#Compute the probabilities for the matrix of first order
for t_1, transition in first_order.items():
    first_order[t_1] = list2pdict(transition)

In [132]:
#Compute the probabilities for the matrix of second order
for key, transition in second_order.items():
    second_order[key] = list2pdict(transition)

In [133]:
def sample_word(dictionary):
    #generate a random probability between 0 and 1
    p0 = np.random.random()
    cumulative = 0
    for transition, probability in dictionary.items():
        #for each transition in the dictionary of the function check if the limit is less than the cumulative
        cumulative += probability
        if p0 < cumulative:
            # If it's less add the transition (word)
            return transition
    #It should always return Something this way we make sure that we are returning always something
    assert(False)

In [134]:
def generate():
    # This function will generate stanzas
    for i in range(4):
        # We're gonna generate stanzas of 4 verses
        sentence = []

        w0 = sample_word(initial)
        sentence.append(w0)
        # Use our previous function to take a word from the initial ones
        # print(first_order[w0])

        w1 = sample_word(first_order[w0])
        sentence.append(w1)
        # We then use again our function to generate a word form the matrix of first states. Notice that it'll look in the key corresponding to the prevous word

        while True:
            # We still iterate over the last dictionary unitl we find the ficticious token END that we've created. In that case the verse will be completed.
            # Until we find the new word we still iterate and making the new words the previous ones
            w2 = sample_word(second_order[(w0,w1)])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [135]:
generate()

i went to bed alone and left me
might just as empty
but it isnt as if and thats not all the money goes so fast
you couldnt call it living for it aint
